In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Model
#print(tf.keras.__version__)
#print(tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

def getColumns(size):
    columns = []
    for i in range(size):
        columns.append(i)
    return columns

directory = "./CMAPSSData/train_FD001.txt"
turbo = pd.read_csv(directory, header=None, sep=' ')
turbo = turbo.drop([26, 27], axis=1)

#for i in turbo.columns:
#    column = turbo[i]
#    print(i)
#    print(column.describe())
    
#print(turbo.values)

# Drop the input parameters to get just the sensors
turboSens = turbo.drop([0,1], axis=1)
turboSens.columns = getColumns(turboSens.shape[1])

# Get the cycles and useful sensors. 
turboUsefulSens = turboSens

min_max_scaler = preprocessing.MinMaxScaler()
norm_sens = min_max_scaler.fit_transform(turboUsefulSens)

turboCycles = turbo[1]
#print(turboCycles)

deteriorate = []
maxCycle = float(turboCycles.max())

currentCount = maxCycle
for i in range(turbo.shape[0] - 1, -1, -1):
    currentRow = turboCycles.loc[i]
    deteriorate.insert(0, currentCount)
    if currentRow == 1:
        currentCount = maxCycle
    else:
        currentCount -= 1

        
deteriorate = pd.DataFrame(deteriorate)

deteriorateSwap = []
for i in deteriorate[0]:
    deteriorateSwap.append(maxCycle - i)
deteriorate = pd.DataFrame(deteriorateSwap)


norm_sens = pd.DataFrame(norm_sens)
deteriorate = deteriorate.join(norm_sens, lsuffix='0', rsuffix='1')
deteriorate.columns = getColumns(deteriorate.shape[1])



Using TensorFlow backend.


In [2]:
array = deteriorate.to_numpy()

arrayCopy = array.tolist()

simulations = []

convCount = 10
convDiv = int(convCount/2)

count = len(array)

while(count > 0):
    cycles = []
    element = [-1,-1]
    while(element[0]!=0):
        element = arrayCopy.pop(0)
        cycles.append(element)
        count-=1
    simulations.append(cycles)
    
#for sim in simulations:
#    lenth = len(sim)
#    remainder = lenth%convCount
#    for i in range(remainder):
#        sim.pop(0)
    
#simsRecomb = []
#for sim in simulations:
#    for i in sim:
#        simsRecomb.append(i)

#print(len(simsRecomb))

In [3]:

tens = []
tensCount = []

for sim in simulations:
    for i in range(convDiv, len(sim) - convDiv):
        ten = []
        total = 0
        for j in range(-convDiv,convDiv):
            index = i + j
            element = sim[index].copy()
            total+= element.pop(0)
            ten.append(element)
        total/=convCount
        tensCount.append(total)
        tens.append(ten)

    
print(len(tens))
print(len(tensCount))

19631
19631


In [4]:
simsReorder = []
simsCycleReorder = []

length = len(tens)
for i in range(length):
    index = rng.randint(0, len(tens)-1)
    simsReorder.append(tens.pop(index))
    simsCycleReorder.append(tensCount.pop(index))
    
    
    
simsTrain = np.array(simsReorder[:15704])
targetTrain = np.array(simsCycleReorder[:15704])

simsTest = np.array(simsReorder[15704:])
targetTest = np.array(simsCycleReorder[15704:])



width = (turboUsefulSens.shape[1])
print(width)

24


In [5]:
visible = layers.Input(shape=(convCount, width,))

cnn1 = layers.Conv1D(150, 3, activation='relu')(visible)
cnn2 = layers.Conv1D(300, 3, activation='relu')(cnn1)
pool = layers.MaxPooling1D(6)(cnn2)
drop = layers.Dropout(0.25)(pool)
flat = layers.Flatten()(pool)

lstm1 = layers.LSTM(150, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(visible)
lstm2 = layers.LSTM(300, dropout=0.1, recurrent_dropout=0.1)(lstm1)

combo = layers.merge.concatenate([flat,lstm2])
dense = layers.Dense(100, activation='relu')(combo)
out = layers.Dense(1)(dense)

model = Model(inputs=visible, outputs=out)




print(model.summary())


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 24)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 8, 150)       10950       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 6, 300)       135300      conv1d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 1, 300)       0           conv1d_2[0][0]                   
____________________________________________________________________________________________

In [6]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='mse', 
              metrics=['mae', 'mse'])

In [14]:
model.fit(simsTrain, targetTrain, epochs=200, verbose=1, validation_data=(simsTest,targetTest))

Train on 15704 samples, validate on 3927 samples
Epoch 1/200
15704/15704 [==============================] - 22s 1ms/step - loss: 3.6857 - mae: 1.4439 - mse: 3.6857 - val_loss: 44.7643 - val_mae: 4.3385 - val_mse: 44.7643
Epoch 2/200
15704/15704 [==============================] - 22s 1ms/step - loss: 3.6461 - mae: 1.4344 - mse: 3.6461 - val_loss: 37.5650 - val_mae: 4.0365 - val_mse: 37.5650
Epoch 3/200
15704/15704 [==============================] - 22s 1ms/step - loss: 3.6826 - mae: 1.4543 - mse: 3.6826 - val_loss: 41.3935 - val_mae: 4.1865 - val_mse: 41.3935
Epoch 4/200
15704/15704 [==============================] - 21s 1ms/step - loss: 3.6546 - mae: 1.4450 - mse: 3.6545 - val_loss: 41.0124 - val_mae: 4.2703 - val_mse: 41.0124
Epoch 5/200
15704/15704 [==============================] - 22s 1ms/step - loss: 3.7533 - mae: 1.4490 - mse: 3.7533 - val_loss: 38.7733 - val_mae: 4.1006 - val_mse: 38.7733
Epoch 6/200
15704/15704 [==============================] - 22s 1ms/step - loss: 3.7778 - ma

In [16]:
model.evaluate(simsTest,targetTest)

3927/3927 [==============================] - 2s 403us/step


[42.110895089347885, 4.470293998718262, 42.11089324951172]

In [26]:
for i in range(100):
    test = np.array([simsTest[i]])
    print(model.predict(test, verbose=1))
    print(targetTest[i])



1/1 [==============================] - 0s 7ms/step
[[150.24297]]
146.5
1/1 [==============================] - 0s 4ms/step
[[62.27805]]
49.5
1/1 [==============================] - 0s 4ms/step
[[168.43832]]
167.5
1/1 [==============================] - 0s 4ms/step
[[86.601]]
87.5
1/1 [==============================] - 0s 4ms/step
[[143.30527]]
152.5
1/1 [==============================] - 0s 3ms/step
[[161.34624]]
154.5
1/1 [==============================] - 0s 4ms/step
[[93.58959]]
83.5
1/1 [==============================] - 0s 4ms/step
[[5.425556]]
5.5
1/1 [==============================] - 0s 3ms/step
[[53.111614]]
52.5
1/1 [==============================] - 0s 2ms/step
[[73.60341]]
71.5
1/1 [==============================] - 0s 2ms/step
[[219.72958]]
211.5
1/1 [==============================] - 0s 3ms/step
[[173.11914]]
173.5
1/1 [==============================] - 0s 2ms/step
[[43.709156]]
45.5
1/1 [==============================] - 0s 2ms/step
[[150.29776]]
138.5
1/1 [================